## Imports and warning filters

We suppress some noisy UFL/Firedrake warnings:

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module=r"ufl\.utils\.sorting")
warnings.filterwarnings("ignore", category=FutureWarning, module=r"firedrake\.interpolation")

from firedrake import *
import finat
import numpy as np
import math, os
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
%matplotlib inline

## Domain and mesh

We use a triangular mesh to enable the KMV element (mass-lumped). Adjust `Lx, Ly, Nx, Ny` as needed:


In [ ]:
Lx, Ly = 4.0, 4.0
Nx, Ny = 100, 100
mesh = RectangleMesh(Nx, Ny, Lx, Ly)
x, y = SpatialCoordinate(mesh)

## Function spaces and mass-lumped quadrature

We use KMV for displacement and a scalar space for auxiliary fields. KMV supports a special quadrature rule that provides mass-lumping on triangular meshes:


In [ ]:
V = VectorFunctionSpace(mesh, "KMV", 2)
V_scalar = FunctionSpace(mesh, "KMV", 2)

deg = V.ufl_element().sub_elements[0].degree()
quad_rule = finat.quadrature.make_quadrature(V.finat_element.cell, deg, "KMV")
dxlump = dx(scheme=quad_rule)

## Time and physical parameters

Choose Lamé parameters $(\lambda, \mu)$ and density $(\rho)$. The P and S speeds are:

$\alpha = \sqrt{(\lambda+2\mu)/\rho}, \quad \beta = \sqrt{\mu/\rho}.$

We print them for convenience:

In [ ]:
T  = 1.0
dt = 0.001
t  = 0.0

rho   = Constant(1.0)
lmbda = Constant(1.0)
mu    = Constant(0.25)

alpha = float(np.sqrt((float(lmbda) + 2.0*float(mu))/float(rho)))
beta  = float(np.sqrt(float(mu)/float(rho)))
print(f"P-wave speed alpha ~ {alpha:.2f}, S-wave speed beta ~ {beta:.2f}")

A_P = Constant(1.0)
A_S = Constant(0.7)

## Unknowns, output files, and helper fields

We set up the Firedrake functions, the `.pvd` for ParaView, and scalar helper fields (`umag`, `P_indicator`, `S_indicator`):

In [ ]:
u     = TrialFunction(V)
v     = TestFunction(V)
u_np1 = Function(V, name="u")
u_n   = Function(V)
u_nm1 = Function(V)

outdir = os.path.join(".", "outputs", "scalar_wave_equation-out")
os.makedirs(outdir, exist_ok=True)
vtk = VTKFile(os.path.join(outdir, "scalar_wave_equation.pvd"))

umag  = Function(V_scalar, name="umag")
P_ind = Function(V_scalar, name="P_indicator")
S_ind = Function(V_scalar, name="S_indicator")

## Source definition

You can choose different source modes. We also use a Gaussian-like spatial weight (`delta_expr`):

In [ ]:
def RickerWavelet(t, freq, amp=1.0):
    t_shifted = t - 1.0/freq
    return amp * (1 - 2*(math.pi*freq*t_shifted)**2) * math.exp(-(math.pi*freq*t_shifted)**2)

def delta_expr(x0, x, y, sigma_x=100.0):
    sigma_x = Constant(sigma_x)
    return exp(-sigma_x * ((x - x0[0])**2 + (y - x0[1])**2))

freq   = 6.0
amp    = 5.0
source = Constant((2.0, 2.0))
ricker = Constant(0.0)

SOURCE_MODE = "radial"

delta_xy = delta_expr(source, x, y)

if SOURCE_MODE == "x":
    def forcing_vec():
        return as_vector((delta_xy*ricker, 0.0))
elif SOURCE_MODE == "radial":
    def forcing_vec():
        gx = delta_xy.dx(0)
        gy = delta_xy.dx(1)
        return as_vector((gx, gy)) * ricker
elif SOURCE_MODE == "tangential":
    ex = (x - source[0]); ey = (y - source[1])
    r = sqrt(ex*ex + ey*ey) + 1e-12
    tx = -ey / r; ty = ex / r
    tvec = as_vector((tx, ty))
    def forcing_vec():
        return tvec * delta_xy * ricker
else:
    raise ValueError("Invalid SOURCE_MODE")

## Variational forms and solver

We use the standard linear isotropic elastic operator:

$ m(u^{n+1}, v) = \frac{\rho}{\Delta t^2} \langle u^{n+1} - 2u^n + u^{n-1}, v \rangle_\text{lumped} $

$ k(u^n, v) = \int \lambda \, \nabla\cdot u^n \; \nabla\cdot v \; dx + 2\mu\, \varepsilon(u^n):\varepsilon(v)\, dx $

The source term is `inner(b_vec, v)`, where `b_vec` depends on the chosen mode:

In [ ]:
def eps(w):
    return 0.5*(grad(w) + grad(w).T)

F_m = (rho/Constant(dt*dt)) * inner(u - 2*u_n + u_nm1, v) * dxlump
F_k = lmbda*div(u_n)*div(v)*dx + 2.0*mu*inner(eps(u_n), eps(v))*dx
b_vec = forcing_vec()
F_s = inner(b_vec, v) * dx

F = F_m + F_k - F_s

A = assemble(lhs(F))
solver = LinearSolver(A, solver_parameters={"ksp_type": "preonly", "pc_type": "jacobi"})

## Analytical and receiver

We maintain your scalar "analytical" expression to compare with the numerical solution at a given receiver point:

In [ ]:
c = Constant(1.5)
Pp = 0.0011

def analitic_solution_2D_green(t):
    r = sqrt((x - source[0])**2 + (y - source[1])**2)
    eps0 = 1e-30
    tP = alpha * t
    phiP = conditional(tP > r,
                       (alpha * A_P / (2.0 * pi)) / sqrt(tP**2 - r**2 + eps0),
                       0.0)
    tS = beta * t
    phiS = conditional(tS > r,
                       (beta * A_S / (2.0 * pi)) / sqrt(tS**2 - r**2 + eps0),
                       0.0)
    return phiP + phiS

receptor_coords = (2.15, 2.0)
u_numerical_history = []
G_history = []
time_points = []

## Time loop

We evolve in time, print elapsed time periodically, store the receiver history, and write VTK files (`u`, `umag`, `P_indicator`, `S_indicator`):

In [ ]:
step = 0
while t < T - 1e-12:
    ricker.assign(RickerWavelet(t, freq, amp=amp))

    R = assemble(rhs(F))
    solver.solve(u_np1, R)

    t += dt
    step += 1
    u_nm1.assign(u_n)
    u_n.assign(u_np1)

    ux_val = float(u_n.at(receptor_coords)[0])
    u_numerical_history.append(ux_val)
    time_points.append(t)

    phi_fun = Function(V_scalar)
    phi_fun.interpolate(analitic_solution_2D_green(t))
    G_history.append(float(phi_fun.at(receptor_coords)))

    if step % 10 == 0:
        umag.interpolate(sqrt(dot(u_n, u_n)))
        P_ind.interpolate(div(u_n))
        S_ind.interpolate(u_n[1].dx(0) - u_n[0].dx(1))
        vtk.write(u_n, umag, P_ind, S_ind, time=t)
        print(f"Elapsed time is: {t:.3f}")

## Postprocessing:

We convolve the recorded analytical history with the Ricker wavelet, compute a relative L2 error, save a comparison plot to `outputs/comparison_plot.png`, and print basic run info:

In [ ]:
def perform_convolution(G_history, R_history, dt):
    from scipy.signal import convolve
    conv_result = convolve(G_history, R_history, mode='full') * dt
    return conv_result[:len(R_history)]

def calculate_L2_error(u_numerical_history, u_analytical_convolved):
    min_len = min(len(u_numerical_history), len(u_analytical_convolved))
    u_num = np.array(u_numerical_history[:min_len])
    u_conv = np.array(u_analytical_convolved[:min_len])
    error_vector = u_num - u_conv
    return np.linalg.norm(error_vector) / (np.linalg.norm(u_conv) + 1e-15)

def plot_comparison(time_points, u_numerical_history, u_analytical_convolved, L2_error, save_path=None):
    min_len = min(len(time_points), len(u_numerical_history), len(u_analytical_convolved))
    time = time_points[:min_len]
    u_num = np.array(u_numerical_history[:min_len])
    u_conv = np.array(u_analytical_convolved[:min_len])
    plt.figure(figsize=(12, 6))
    plt.plot(time, u_num,  label='Numerical', linewidth=2)
 #   plt.plot(time, u_conv, '--', label='Analytical', linewidth=2.5, alpha=0.8)
    plt.title('')
    plt.xlabel('Time (s)'); plt.ylabel('Amplitude')
    plt.legend(loc='lower right'); plt.grid(True, linestyle=':', alpha=0.6)
    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=200)
    plt.close()

R_history = np.array([RickerWavelet(ti, freq, amp=amp) for ti in time_points])
u_analytical_convolved = perform_convolution(G_history, R_history, dt)
L2_error = calculate_L2_error(u_numerical_history, u_analytical_convolved)
plot_path = os.path.join(outdir, "comparison_plot.png")
plot_comparison(time_points, u_numerical_history, u_analytical_convolved, L2_error, save_path=plot_path)